# 5교시: Tool을 이용한 검색 Agent 제작

## 실습 목표
- Function Calling 원리 이해
- 검색 도구 연동 Agent 구현
- 나만의 Agent 만들기

---
## 1. 환경 설정

In [ ]:
# 필요한 패키지 설치
!pip install langchain langchain-google-genai langchain-community google-generativeai duckduckgo-search -q

In [ ]:
# 라이브러리 임포트
import os
import json
from getpass import getpass
from datetime import datetime

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool

print("라이브러리 로드 완료!")

In [ ]:
# API Key 설정
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "")
if not GOOGLE_API_KEY:
    GOOGLE_API_KEY = getpass("Google API Key: ")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Gemini 설정
genai.configure(api_key=GOOGLE_API_KEY)

# Tavily API Key (선택사항 - 없으면 DuckDuckGo 사용)
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY", "")
if not TAVILY_API_KEY:
    print("Tavily API Key가 없습니다. DuckDuckGo 검색을 사용합니다.")
    
if TAVILY_API_KEY:
    os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

print("API Key 설정 완료!")

---
## 2. Function Calling 이해하기

### 2.1 기본 원리

In [ ]:
# Gemini Function Calling 기본 예제

# 도구(함수) 정의 - Gemini 형식
get_weather_func = genai.protos.FunctionDeclaration(
    name="get_weather",
    description="특정 도시의 현재 날씨를 조회합니다. 날씨 관련 질문에 사용하세요.",
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties={
            "city": genai.protos.Schema(
                type=genai.protos.Type.STRING,
                description="도시 이름 (예: 서울, 부산, 제주)"
            )
        },
        required=["city"]
    )
)

calculate_func = genai.protos.FunctionDeclaration(
    name="calculate",
    description="수학 계산을 수행합니다. 계산이 필요한 질문에 사용하세요.",
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties={
            "expression": genai.protos.Schema(
                type=genai.protos.Type.STRING,
                description="계산할 수식 (예: 2 + 2, 100 * 5)"
            )
        },
        required=["expression"]
    )
)

# 도구 모음
tools_for_gemini = genai.protos.Tool(
    function_declarations=[get_weather_func, calculate_func]
)

print("도구 정의 완료!")
print(f"- get_weather: {get_weather_func.description}")
print(f"- calculate: {calculate_func.description}")

In [ ]:
# 실제 함수 구현 (시뮬레이션)
def get_weather(city: str) -> str:
    """날씨 정보 반환 (실제로는 API 호출)"""
    # 실제 구현에서는 날씨 API 호출
    weather_data = {
        "서울": "맑음, 기온 15°C, 습도 45%",
        "부산": "흐림, 기온 18°C, 습도 60%",
        "제주": "구름 많음, 기온 20°C, 습도 70%"
    }
    return weather_data.get(city, f"{city}의 날씨 정보를 찾을 수 없습니다.")

def calculate(expression: str) -> str:
    """수식 계산"""
    try:
        result = eval(expression)
        return str(result)
    except:
        return "계산 오류"

# 함수 매핑
available_functions = {
    "get_weather": get_weather,
    "calculate": calculate
}

In [ ]:
# Function Calling 전체 흐름 (Gemini)
model = genai.GenerativeModel(
    model_name='gemini-2.5-flash-lite',
    tools=[tools_for_gemini]
)

def chat_with_tools(user_message):
    print(f"\n[사용자] {user_message}")
    
    # 1단계: LLM에게 질문
    response = model.generate_content(user_message)
    
    # 2단계: 도구 호출이 필요한지 확인
    if response.candidates[0].content.parts[0].function_call:
        function_call = response.candidates[0].content.parts[0].function_call
        function_name = function_call.name
        function_args = dict(function_call.args)
        
        print("\n[LLM] 도구 호출 결정:")
        print(f"  - 도구: {function_name}")
        print(f"  - 인자: {function_args}")
        
        # 3단계: 실제 함수 실행
        function_response = available_functions[function_name](**function_args)
        print(f"  - 결과: {function_response}")
        
        # 4단계: 결과를 모델에 전달하여 최종 응답 생성
        response2 = model.generate_content([
            user_message,
            response.candidates[0].content,
            genai.protos.Content(
                parts=[genai.protos.Part(
                    function_response=genai.protos.FunctionResponse(
                        name=function_name,
                        response={"result": function_response}
                    )
                )]
            )
        ])
        answer = response2.text
    else:
        # 도구 호출 없이 바로 응답
        answer = response.text
    
    print(f"\n[AI] {answer}")
    return answer

In [ ]:
# 테스트
chat_with_tools("서울 날씨 어때?")

In [ ]:
chat_with_tools("123 곱하기 456은?")

In [ ]:
chat_with_tools("안녕하세요!")

---
## 3. 검색 Agent 만들기

### 3.1 검색 도구 설정

In [ ]:
# DuckDuckGo 검색 도구 (무료, API 키 불필요)
from langchain_community.tools import DuckDuckGoSearchRun

# 검색 도구 생성
search_tool = DuckDuckGoSearchRun()

# 테스트 검색
print("[DuckDuckGo 검색 테스트]")
result = search_tool.invoke("2024년 노벨상 수상자")
print(result[:500])

In [ ]:
# Tavily 검색 (더 정확, API 키 필요)
if TAVILY_API_KEY:
    from langchain_community.tools.tavily_search import TavilySearchResults
    
    tavily_tool = TavilySearchResults(
        max_results=3,
        search_depth="basic"
    )
    
    print("[Tavily 검색 테스트]")
    result = tavily_tool.invoke("2024년 AI 트렌드")
    print(result)
else:
    print("Tavily API Key가 없어 DuckDuckGo를 사용합니다.")

### 3.2 커스텀 도구 만들기

In [ ]:
# @tool 데코레이터로 커스텀 도구 만들기

@tool
def get_current_time() -> str:
    """현재 날짜와 시간을 반환합니다. 시간 관련 질문에 사용하세요."""
    now = datetime.now()
    return now.strftime("%Y년 %m월 %d일 %H시 %M분")

@tool
def calculator(expression: str) -> str:
    """수학 계산을 수행합니다.
    
    Args:
        expression: 계산할 수식 (예: 2 + 2, 100 * 5, 2 ** 10)
    """
    try:
        # 안전한 계산만 허용
        allowed_chars = set('0123456789+-*/().** ')
        if not all(c in allowed_chars for c in expression):
            return "허용되지 않는 문자가 포함되어 있습니다."
        result = eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"계산 오류: {str(e)}"

@tool
def korean_to_english(text: str) -> str:
    """한국어를 영어로 번역합니다.
    
    Args:
        text: 번역할 한국어 텍스트
    """
    # Gemini로 번역
    translate_model = genai.GenerativeModel('gemini-2.5-flash-lite')
    response = translate_model.generate_content(
        f"Translate the following Korean text to English. Only provide the translation:\n{text}"
    )
    return response.text

print("커스텀 도구 정의 완료!")
print(f"- get_current_time: {get_current_time.description}")
print(f"- calculator: {calculator.description}")
print(f"- korean_to_english: {korean_to_english.description}")

In [ ]:
# 도구 테스트
print("현재 시간:", get_current_time.invoke({}))
print("계산:", calculator.invoke({"expression": "2 ** 10"}))

### 3.3 LangChain Agent 구성

In [ ]:
# 도구 목록
tools = [
    search_tool,
    get_current_time,
    calculator,
    korean_to_english
]

# LLM 설정 (Gemini)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0,
    google_api_key=GOOGLE_API_KEY
)

# 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 유용한 AI 어시스턴트입니다.

다음 도구들을 사용할 수 있습니다:
- 웹 검색: 최신 정보나 사실 확인이 필요할 때
- 현재 시간: 날짜/시간 관련 질문
- 계산기: 수학 계산이 필요할 때
- 번역: 한국어를 영어로 번역할 때

항상 한국어로 친절하게 답변하세요.
불확실한 정보는 검색을 통해 확인하세요."""),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Agent 생성 (tool_calling_agent 사용 - Gemini 호환)
agent = create_tool_calling_agent(llm, tools, prompt)

# Agent 실행기 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,  # 실행 과정 출력
    handle_parsing_errors=True
)

print("Agent 생성 완료!")

In [ ]:
# Agent 테스트
result = agent_executor.invoke({"input": "지금 몇 시야?"})
print("\n최종 답변:", result["output"])

In [ ]:
# 검색 테스트
result = agent_executor.invoke({"input": "최근 OpenAI 관련 뉴스 알려줘"})
print("\n최종 답변:", result["output"])

In [ ]:
# 복합 질문 테스트
result = agent_executor.invoke({"input": "2의 20승이 얼마야? 그리고 그 결과를 영어로 말해줘"})
print("\n최종 답변:", result["output"])

---
## 4. Streamlit Agent 앱 만들기

In [ ]:
%%writefile agent_chatbot.py
"""
Tool Agent 챗봇 (Gemini 버전)
실행: streamlit run agent_chatbot.py
"""
import streamlit as st
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool
from datetime import datetime
import os

st.set_page_config(page_title="AI Agent", page_icon="🤖")

st.title("AI Agent 챗봇 🤖")
st.caption("검색, 계산, 시간 확인이 가능한 AI 에이전트입니다!")

# 사이드바
with st.sidebar:
    api_key = st.text_input("Google API Key", type="password")
    
    st.divider()
    st.markdown("### 사용 가능한 도구")
    st.markdown("- 🔍 웹 검색")
    st.markdown("- 🕐 현재 시간")
    st.markdown("- 🧮 계산기")
    
    st.divider()
    st.markdown("### 질문 예시")
    st.markdown("- 오늘 AI 관련 뉴스 알려줘")
    st.markdown("- 지금 몇 시야?")
    st.markdown("- 123 * 456 계산해줘")
    
    if st.button("대화 초기화"):
        st.session_state.messages = []
        st.rerun()

# 도구 정의
@tool
def get_current_time() -> str:
    """현재 날짜와 시간을 반환합니다."""
    return datetime.now().strftime("%Y년 %m월 %d일 %H시 %M분")

@tool
def calculator(expression: str) -> str:
    """수학 계산을 수행합니다. Args: expression: 계산할 수식"""
    try:
        return str(eval(expression))
    except:
        return "계산 오류"

# Agent 초기화
@st.cache_resource
def init_agent(_api_key):
    os.environ["GOOGLE_API_KEY"] = _api_key
    
    tools = [
        DuckDuckGoSearchRun(),
        get_current_time,
        calculator
    ]
    
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-lite",
        temperature=0,
        google_api_key=_api_key
    )
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """당신은 유용한 AI 어시스턴트입니다.
웹 검색, 시간 확인, 계산 도구를 사용할 수 있습니다.
항상 한국어로 친절하게 답변하세요."""),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ])
    
    agent = create_tool_calling_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=False)

# 메시지 초기화
if "messages" not in st.session_state:
    st.session_state.messages = []

# 이전 메시지 표시
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# 새 메시지 처리
if prompt := st.chat_input("무엇이든 물어보세요..."):
    if not api_key:
        st.error("API Key를 입력해주세요!")
        st.stop()
    
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)
    
    with st.chat_message("assistant"):
        with st.spinner("생각 중..."):
            try:
                agent_executor = init_agent(api_key)
                result = agent_executor.invoke({"input": prompt})
                answer = result["output"]
                st.markdown(answer)
                st.session_state.messages.append({"role": "assistant", "content": answer})
            except Exception as e:
                st.error(f"오류: {str(e)}")

---
## 5. 나만의 Agent 만들기 프로젝트

### 5.1 기획서 템플릿

In [ ]:
# 기획서 템플릿
project_template = """
# Agent 기획서

## 1. 기본 정보
- Agent 이름: 
- 팀명/팀원: 
- 역할/목적: 

## 2. 사용 도구
- [ ] 웹 검색
- [ ] 현재 시간
- [ ] 계산기
- [ ] 커스텀 도구 1: 
- [ ] 커스텀 도구 2: 

## 3. 시스템 프롬프트
(Agent의 역할과 성격을 정의)

## 4. 예상 대화 시나리오
사용자: 
Agent: 

사용자: 
Agent: 

## 5. 기대 효과
"""

print(project_template)

### 5.2 예시: 여행 플래너 Agent

In [ ]:
# 여행 플래너 Agent 예시

@tool
def search_tourist_spots(location: str) -> str:
    """특정 지역의 관광지를 검색합니다.
    
    Args:
        location: 검색할 지역 (예: 서울, 제주, 부산)
    """
    search = DuckDuckGoSearchRun()
    return search.invoke(f"{location} 인기 관광지 추천")

@tool
def search_restaurants(location: str, food_type: str = "") -> str:
    """특정 지역의 맛집을 검색합니다.
    
    Args:
        location: 검색할 지역
        food_type: 음식 종류 (선택사항)
    """
    search = DuckDuckGoSearchRun()
    query = f"{location} 맛집 추천"
    if food_type:
        query += f" {food_type}"
    return search.invoke(query)

# 여행 플래너 Agent 구성
travel_tools = [
    search_tool,
    search_tourist_spots,
    search_restaurants,
    get_current_time
]

travel_prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 친절한 여행 플래너 AI입니다.

역할:
- 여행 계획 수립을 도와줍니다
- 관광지, 맛집 정보를 검색하여 추천합니다
- 여행 일정을 구체적으로 제안합니다

스타일:
- 친근하고 밝은 톤으로 대화합니다
- 이모지를 적절히 사용합니다
- 구체적인 정보와 함께 개인적인 팁도 제공합니다

항상 한국어로 답변하세요."""),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

travel_agent = create_tool_calling_agent(llm, travel_tools, travel_prompt)
travel_executor = AgentExecutor(agent=travel_agent, tools=travel_tools, verbose=True)

print("여행 플래너 Agent 생성 완료!")

In [ ]:
# 여행 플래너 테스트
result = travel_executor.invoke({"input": "제주도 2박3일 여행 계획 세워줘"})
print("\n최종 답변:")
print(result["output"])

### 5.3 나만의 Agent 구현하기

In [ ]:
# TODO: 여기에 나만의 Agent를 구현하세요!

# 1. 커스텀 도구 정의
@tool
def my_custom_tool(query: str) -> str:
    """나만의 도구 설명
    
    Args:
        query: 입력 파라미터 설명
    """
    # 도구 구현
    return "결과"

# 2. 도구 목록
my_tools = [
    search_tool,
    # my_custom_tool,
]

# 3. 시스템 프롬프트
my_system_prompt = """
당신은 [역할]입니다.

역할:
- 

스타일:
- 
"""

# 4. Agent 생성
# my_prompt = ChatPromptTemplate.from_messages([...])
# my_agent = create_openai_functions_agent(llm, my_tools, my_prompt)
# my_executor = AgentExecutor(agent=my_agent, tools=my_tools)

# 5. 테스트
# result = my_executor.invoke({"input": "테스트 질문"})
# print(result["output"])

---
## 정리

### 오늘 배운 내용

1. **Function Calling**: LLM이 도구를 선택하고 호출하는 원리
2. **Tool 정의**: name, description, parameters
3. **LangChain Agent**: 도구 + LLM + 실행기
4. **커스텀 Tool**: @tool 데코레이터 사용법

### 전체 수업 요약

| 차시 | 핵심 내용 |
|------|----------|
| 1-2교시 | LLM 원리/한계 → Agentic AI 필요성 |
| 3교시 | Streamlit + API = 웹앱 |
| 4교시 | RAG = 검색 + LLM |
| 5교시 | Tool = LLM의 행동 능력 |
